## ML Pipelines with ZenML  

**Key Concepts:** *ML Pipelines, Steps*  

- In this notebook, techniques are shown on how to easily convert existing ML code into ML pipelines using ZenML.  
- Models will be built with [sklearn](https://scikit-learn.org).  
- ZenML & sklearn integration installed. Use the following command, which will also restart the kernel of your notebook:  

```bash
# Install ZenML with server extras
python -m pip install "zenml[server]"

# Install sklearn integration for ZenML
python -m zenml integration install sklearn -y


In [1]:
import IPython